In [3]:
from selenium import webdriver
import pandas as pd
import re
from bs4 import BeautifulSoup
import requests
options = webdriver.ChromeOptions()
options.add_argument('headless')
driver = webdriver.Chrome(chrome_options=options)

search=pd.read_excel('output.xlsx',sheet_name='Sheet2')
search=search['urls'].head(25)
product=[]
product_list=[]
product_details=[]
quantity=[]
rating=[]
Category=[]
Sub_Category=[]
Sub_Category2=[]
total_reviews=[]
total_ratings=[]
price=[]

def printurl(url,j):
    if j<5: #No of pages to be scrapped for each catgeory
        driver.get(url)
        page = requests.get(url)
        soup=BeautifulSoup(page.content, 'html.parser')
        name=[]
        for i in soup.find_all('h2'):
            name.append(i['title'])
        Featured=[]
        for i in soup.find_all('div',class_= 'tags-offer-container'):
            if i.find('span',class_='featured'):
                Featured.append("Yes")
            else:
                Featured.append("No")
        links=[]
        productId=[]
        categoryId=[]
        for l in soup.find_all('div',class_='product-list-box'):
            link=l.find('a')['href']
            if(l.find('h2') is not None):
                k="https://www.nykaa.com"+link
                links.append(k)
                try:
                    c=k.split("skuId")[1].replace("=","").split("&")[0]
                    productId.append(c)  
                except:
                    productId.append('-') 
                try:
                    c=k.split("categoryId")[1].replace("=","").split("&")[0]
                    categoryId.append(c)  
                except:
                    categoryId.append('-') 

                    
        discount=[]    
        for s in soup.find_all('div',class_='m-content__product-list__price'):
            if s.find('div',class_='discount-info'):
                discount.append(s.find('div',class_='discount-info').find('span').text)
            else:
                discount.append("-")
        product.append(list(zip(productId,categoryId,name,Featured,links,discount)))
        try:
            lis=driver.find_element_by_css_selector('li.next a')
            next_url=lis.get_attribute('href')
            printurl(next_url,j+1)
            print(j)
        except:
            pass
        
for u in search:
    printurl(u,0)
    for i in product:
        for j in i:
            product_list.append(j)
            product=[]
print(len(product_list))

#Second page scrapping
def getdetails2(url):
    try:
        driver.get(url)
        page = requests.get(url)
        soup=BeautifulSoup(page.content, 'html.parser')
        try:
            quantity.append(driver.find_element_by_css_selector("span.shade-txt").text.replace('(','').replace(')',''))    
        except:
            quantity.append("-")
        try:
            price.append(int(soup.find("span",class_="post-card__content-price-offer").text.replace("₹","")))
        except:
            price.append("-")
        try:
            rating.append(soup.find("div",class_="m-content__product-list__ratings").find("meta")['content'])
        except:
            rating.append("-")
        try:
            total=int(soup.find("div",class_="m-content__product-list__ratings").find_all("meta")[2]['content'])
        except:
            total=0
        total_reviews.append(total)
        try:
            total=int(soup.find("div",class_="rating-count-popup").find("meta")['content'])
        except:
            total=0
        total_ratings.append(total)
        try:
            lis=driver.find_elements_by_tag_name("ol li")
            cat=[]
            for i in lis:
                k=i.find_elements_by_tag_name("a")
                for j in k:
                    cat.append(j.get_attribute('innerHTML'))
            if len(cat)>=2:
                Category.append(cat[1])
            else:
                Category.append("-")
            if len(cat)>=3:
                Sub_Category.append(cat[2])
            else:
                Sub_Category.append("-")
            if len(cat)>=4:
                Sub_Category2.append(cat[3])
            else:
                Sub_Category2.append("-")
        except:
            Category.append("-")
            Sub_Category.append("-")
            Sub_Category2.append("-")
    except:
        quantity.append("-")
        price.append("-")
        rating.append("-")
        total_reviews.append(0)
        total_ratings.append(0)
        Category.append("-")
        Sub_Category.append("-")
        Sub_Category2.append("-")
        
quantity=[]
rating=[]
Category=[]
Sub_Category=[]
Sub_Category2=[]
total_reviews=[]
total_ratings=[]
price=[]
t=0
for k in product_list:
    getdetails2(k[3])
    print(t)
    t=t+1

product_details.append(list(zip(price,quantity,rating,total_reviews,total_ratings,Category,Sub_Category,Sub_Category2)))
product_specs=[]
for i in product_details:
    for j in i:
        product_specs.append(j)
product_specs=[]
for i in product_details:
    for j in i:
        product_specs.append(j)

results=[]
for a,b in zip(product_list,product_specs):
    results.append(a+b)
df1= pd.DataFrame(results, columns =['Product_Id','Category_Id','Name', 'Featured','Url','Discount', 'Marked Price','Quantity','Rating(out of 5)','Review Count','Rating Count','Category','SubCategory1','SubCategory2'])
print(df1)

<ipython-input-3-2417888062d4>:8: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=options)


0
Empty DataFrame
Columns: [Product_Id, Category_Id, Name, Featured, Url, Discount, Marked Price, Quantity, Rating(out of 5), Review Count, Rating Count, Category, SubCategory1, SubCategory2]
Index: []


In [ ]:
search=search['urls'].tail(25)
product=[]
product_list=[]
product_details=[]
quantity=[]
rating=[]
Category=[]
Sub_Category=[]
Sub_Category2=[]
total_reviews=[]
total_ratings=[]
price=[]

for u in search:
    printurl(u,0)
    for i in product:
        for j in i:
            product_list.append(j)
            product=[]
print(len(product_list))

quantity=[]
rating=[]
Category=[]
Sub_Category=[]
Sub_Category2=[]
total_reviews=[]
total_ratings=[]
price=[]
t=0
for k in product_list:
    getdetails2(k[3])
    print(t)
    t=t+1

product_details.append(list(zip(price,quantity,rating,total_reviews,total_ratings,Category,Sub_Category,Sub_Category2)))
product_specs=[]
for i in product_details:
    for j in i:
        product_specs.append(j)
product_specs=[]
for i in product_details:
    for j in i:
        product_specs.append(j)

results=[]
for a,b in zip(product_list,product_specs):
    results.append(a+b)
df2= pd.DataFrame(results, columns =['Product_Id','Name', 'Featured','Url','Discount', 'Marked Price','Quantity','Rating(out of 5)','Review Count','Rating Count','Category','SubCategory1','SubCategory2'])
print(df2)


In [ ]:
frames = [df1, df2]
df = pd.concat(frames)
df.reset_index(drop=True, inplace=True)

In [ ]:
#Brand_n_Ingredient_categorisation
df["Name"]=df["Name"].str.lower()
df["Name"]=df["Name"].str.replace(".","")
brand=pd.read_excel('brands.xlsx')
brand_list=brand['Brand'].to_list()
for i in brand_list:
    i.replace(".","")
def matcher(x):
    for i in ing_list:
        if i.lower() in x.lower():
            return i
    else:
        return np.nan
word_list = brand_list
pat = '|'.join(r"\b{}\b".format(x) for x in word_list)
df1 = df['Name'].str.lower().str.extractall('(' + pat + ')', flags = re.I)[0].unstack().add_prefix('Brand_')
df_col = pd.concat([df,df1], axis=1)
ingredient=pd.read_excel('ingredient_list.xlsx')
ing_list=ingredient['Ingredients'].to_list()
word_list2 = ing_list
pat = '|'.join(r"\b{}\b".format(x) for x in word_list2)
df2 = df['Name'].str.lower().str.extractall('(' + pat + ')', flags = re.I)[0].unstack().add_prefix('Ingredient_')
df_col_2 = pd.concat([df_col,df2], axis=1)

from datetime import date
today =  str(date.today())
df_col_2.to_excel("Nykaa "+today+".xlsx")